In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import copy

### 그림그리는 함수

In [4]:
def show_v_table_small(v_table, env):
    for i in range(env.reward.shape[0]):        
        print("+----------"*env.reward.shape[1])
        print("|", end="")
        for j in range(env.reward.shape[1]):
            print("{0:8.2f}  |".format(v_table[i,j]),end="")
        print()
    print("+----------"*env.reward.shape[1])

# V table 그리기    
def show_v_table(v_table, env):    
    for i in range(env.reward.shape[0]):        
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    print("                 |",end="")
                if k==1:
                        print("   {0:8.2f}      |".format(v_table[i,j]),end="")
                if k==2:
                    print("                 |",end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")
    
# Q table 그리기
def show_q_table(q_table,env):
    for i in range(env.reward.shape[0]):
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    print("{0:10.2f}       |".format(q_table[i,j,0]),end="")
                if k==1:
                    print("{0:6.2f}    {1:6.2f} |".format(q_table[i,j,3],q_table[i,j,1]),end="")
                if k==2:
                    print("{0:10.2f}       |".format(q_table[i,j,2]),end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")
    

# 정책 policy 화살표로 그리기
def show_q_table_arrow(q,env):
    for i in range(env.reward.shape[0]):        
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    if np.max(q[i,j,:]) == q[i,j,0]:
                        print("        ↑       |",end="")
                    else:
                        print("                 |",end="")
                if k==1:                    
                    if np.max(q[i,j,:]) == q[i,j,1] and np.max(q[i,j,:]) == q[i,j,3]:
                        print("      ←  →     |",end="")
                    elif np.max(q[i,j,:]) == q[i,j,1]:
                        print("          →     |",end="")
                    elif np.max(q[i,j,:]) == q[i,j,3]:
                        print("      ←         |",end="")
                    else:
                        print("                 |",end="")
                if k==2:
                    if np.max(q[i,j,:]) == q[i,j,2]:
                        print("        ↓       |",end="")
                    else:
                        print("                 |",end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")    
    
# 정책 policy 화살표로 그리기
def show_policy_small(policy,env):
    for i in range(env.reward.shape[0]):        
        print("+----------"*env.reward.shape[1],end="")
        print("+")
        print("|", end="")
        for j in range(env.reward.shape[1]):
            if env.reward_list1[i][j] == "road":
                if policy[i,j] == 0:
                    print("   ↑     |",end="")
                elif policy[i,j] == 1:
                    print("   →     |",end="")
                elif policy[i,j] == 2:
                    print("   ↓     |",end="")
                elif policy[i,j] == 3:
                    print("   ←     |",end="")
            else:
                print("          |",end="")
        print()
    print("+----------"*env.reward.shape[1],end="")
    print("+")
    
# 정책 policy 화살표로 그리기
def show_policy(policy,env):
    for i in range(env.reward.shape[0]):        
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    print("                 |",end="")
                if k==1:
                    if policy[i,j] == 0:
                        print("      ↑         |",end="")
                    elif policy[i,j] == 1:
                        print("      →         |",end="")
                    elif policy[i,j] == 2:
                        print("      ↓         |",end="")
                    elif policy[i,j] == 3:
                        print("      ←         |",end="")
                if k==2:
                    print("                 |",end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")

### Agent 구현

In [5]:
class Agent():
    
    # 1. 행동에 따른 에이전트의 좌표 이동(위, 오른쪽, 아래, 왼쪽) 
    action = np.array([[-1,0],[0,1],[1,0],[0,-1]])
    
    # 2. 각 행동별 선택확률
    select_action_pr = np.array([0.25,0.25,0.25,0.25])
    
    # 3. 에이전트의 초기 위치 저장
    def __init__(self):
        self.pos = (0,0)
    
    # 4. 에이전트의 위치 저장
    def set_pos(self,position):
        self.pos = position
        return self.pos
    
    # 5. 에이전트의 위치 불러오기
    def get_pos(self):
        return self.pos

### Environment 구현

In [6]:
class Environment():
    
    # 1. 미로밖(절벽), 길, 목적지와 보상 설정
    cliff = -3
    road = -1
    goal = 1
    
    # 2. 목적지 좌표 설정
    goal_position = [2,2]
    
    # 3. 보상 리스트 숫자
    reward_list = [[road,road,road],
                   [road,road,road],
                   [road,road,goal]]
    
    # 4. 보상 리스트 문자
    reward_list1 = [["road","road","road"],
                    ["road","road","road"],
                    ["road","road","goal"]]
    
    # 5. 보상 리스트를 array로 설정
    def __init__(self):
        self.reward = np.asarray(self.reward_list)    

    # 6. 선택된 에이전트의 행동 결과 반환 (미로밖일 경우 이전 좌표로 다시 복귀)
    def move(self, agent, action):
        
        done = False
        
        # 6.1 행동에 따른 좌표 구하기
        new_pos = agent.pos + agent.action[action]
        
        # 6.2 현재좌표가 목적지 인지확인
        if self.reward_list1[agent.pos[0]][agent.pos[1]] == "goal":
            reward = self.goal
            observation = agent.set_pos(agent.pos)
            done = True
        # 6.3 이동 후 좌표가 미로 밖인 확인    
        elif new_pos[0] < 0 or new_pos[0] >= self.reward.shape[0] or new_pos[1] < 0 or new_pos[1] >= self.reward.shape[1]:
            reward = self.cliff
            observation = agent.set_pos(agent.pos)
            done = True
        # 6.4 이동 후 좌표가 길이라면
        else:
            observation = agent.set_pos(new_pos)
            reward = self.reward[observation[0],observation[1]]
            
        return observation, reward, done

### TD(0) prediction (복습, 참고용)

In [7]:
# TD(0) prediction
np.random.seed(0)
# 환경, 에이전트를 초기화
env = Environment()
agent = Agent()
gamma = 0.9

# 초기화 : 
# π← 평가할 정책
# 가능한 모든 행동이 무작위로 선택되도록 지정
# 𝑉← 임의의 상태가치 함수
V = np.zeros((env.reward.shape[0], env.reward.shape[1]))

# 최대 에피소드, 에피소드의 최대 길이를 지정
max_episode = 10000
max_step = 100

# alpha: 스텝 사이즈
# gamma: 감마값, 할인율
alpha = 0.01

print("start TD(0) prediction")

# 각 에피소드에 대해 반복 :
for epi in tqdm(range(max_episode)):
    delta =0
    
    # s 를 초기화
    i = 0
    j = 0
    agent.set_pos([i,j])

    #  에피소드의 각 스텝에 대해 반복 :
    for k in range(max_step):
        pos = agent.get_pos()
        
        # a←상태 𝑠 에서 정책 π에 의해 결정된 행동 
        # 가능한 모든 행동이 무작위로 선택되게 함
        action = np.random.randint(0,4)
        
        # 행동 a 를 취한 후 보수 r과 다음 상태 s’를 관측
        # s←𝑠'
        observation, reward, done = env.move(agent,action)
        
        # V(𝑠)←V(𝑠)+ α[𝑟+𝛾𝑉(𝑠^)−𝑉(𝑠)]
        V[pos[0],pos[1]] += alpha * (reward + gamma * V[observation[0],observation[1]] - V[pos[0],pos[1]])
        
        # s가 마지막 상태라면 종료
        if done == True:
            break
            
print("V(s)")
show_v_table(np.round(V,2),env)

start TD(0) prediction


100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 37885.81it/s]

V(s)
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|     -12.59      |     -11.01      |     -10.12      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|     -10.85      |      -8.53      |      -5.80      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -9.68      |      -6.06      |       3.48      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+


`도착지점으로 도착할수록 값이 커짐`

### 𝜖−𝑔𝑟𝑒𝑒𝑑𝑦, 𝑔𝑟𝑒𝑒𝑑𝑦  함수 작성

* 각 상태의 행동가치 Q(s, a)가 저장된 Q-table과 에이전트 객체, 입실론 값을 입력받아 각 정책에 맞게 선택하고 선택된 행동 반환

In [8]:
#  𝜖−𝑔𝑟𝑒𝑒𝑑𝑦  함수 작성
def e_greedy(Q_table,agent,epsilon):
    pos = agent.get_pos()
    
    # np.argmax: 최댓값 반환
    greedy_action = np.argmax(Q_table[pos[0],pos[1],:])
    
    # [0, 0, 0, 0]
    pr = np.zeros(4)
    for i in range(len(agent.action)):
        if i == greedy_action:
            pr[i] = 1 - epsilon + epsilon/len(agent.action)
        else:
            pr[i] = epsilon / len(agent.action)

    return np.random.choice(range(0,len(agent.action)), p=pr)    

# 𝑔𝑟𝑒𝑒𝑑𝑦 함수 작성
def greedy(Q_table,agent,epsilon):
    pos = agent.get_pos()
    
    # 최적의 행동만 선택되고, 이외의 행동은 절대 선택되지 않음
    return np.argmax(Q_table[pos[0],pos[1],:])   

### TD(0) coltrol : SARSA

In [10]:
# TD(0) control : SARSA
np.random.seed(0)

# 환경, 에이전트를 초기화
env = Environment()
agent = Agent()
gamma = 0.9

# 모든 𝑠∈𝑆,𝑎∈𝐴(𝑆)에 대해 초기화:
# 𝑄(𝑠,𝑎)←임의의 값
Q_table = np.random.rand(env.reward.shape[0], env.reward.shape[1],len(agent.action))

# Q(𝑡𝑒𝑟𝑚𝑖𝑛𝑎𝑙−𝑠𝑡𝑎𝑡𝑒,𝑎)=0
Q_table[2,2,:] = 0

max_episode = 10000
max_step = 100

print("start TD(0) control : SARSA")
alpha = 0.1
epsilon = 0.8

# 각 에피소드에 대해서 반복 :
for epi in tqdm(range(max_episode)):
    
    # 델타 0 -> 변화량 0
    dleta = 0
    
    # S 를 초기화
    i = 0
    j = 0
    agent.set_pos([i,j])
    temp =0
    
    # s 에서 행동 정책(Behavior policy)으로 행동 a를 선택 ( 예 : ε-greedy
    action = e_greedy(Q_table,agent,epsilon)

    # 에피소드의 각 스텝에 대해서 반복 :
    for k in range(max_step):
        pos = agent.get_pos()
        
        # Q_visit_count[action, pos[0],pos[1]] +=1
        # 행동 a 를 취한 후 보상 r과 다음 상태 s^'  관측
        observation, reward, done = env.move(agent, action)

        # s^' 에서 타깃 정책(Target policy)으로 행동 a^'를 선택 (예: ε-greedy)
        next_action = e_greedy(Q_table,agent,epsilon)
        
        # Q(𝑆,𝐴)←Q(𝑆,𝐴) + α[𝑅+𝛾𝑄(𝑆',𝐴')−𝑄(𝑆,𝐴)]
        Q_table[pos[0],pos[1],action] += alpha * (reward + gamma * Q_table[observation[0], observation[1],next_action] - Q_table[pos[0],pos[1],action])

        # s←s^' ; a←a^'
        action = next_action
        
        # s가 마지막 상태라면 종료
        if done == True:
            break
        
# 학습된 정책에서 최적 행동 추출
optimal_policy = np.zeros((env.reward.shape[0], env.reward.shape[1]))
for i in range(env.reward.shape[0]):
    for j in range(env.reward.shape[1]):
        optimal_policy[i,j] = np.argmax(Q_table[i,j,:])

print("SARSA : Q(s,a)")
show_q_table(np.round(Q_table,2),env)
print("SARSA :optimal policy")
show_policy(optimal_policy,env)

start TD(0) control : SARSA


100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 5617.87it/s]

SARSA : Q(s,a)
+-----------------+-----------------+-----------------+
|     -9.20       |     -7.30       |     -5.16       |
| -9.71     -5.47 | -7.55     -4.38 | -5.91     -6.55 |
|     -5.24       |     -3.63       |      0.40       |
+-----------------+-----------------+-----------------+
|     -6.74       |     -4.63       |     -4.46       |
| -7.34     -3.18 | -5.42      3.59 | -3.82     -1.42 |
|     -3.02       |     -1.13       |      9.70       |
+-----------------+-----------------+-----------------+
|     -5.06       |     -3.86       |      9.63       |
| -6.74     -1.92 | -4.77      9.68 |  9.66      9.68 |
|     -5.29       |     -1.65       |      9.69       |
+-----------------+-----------------+-----------------+
SARSA :optimal policy
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      ↓         |      ↓         |      ↓         |
|                 |                 |                 |
+-------------

`미로에서 최적 행동을 학습함 (행동가치가 가장 높은 방향)`

### TD(0) control : Q-learning

In [11]:
#  TD(0) contro : Q-learning

# 환경, 에이전트를 초기화
env = Environment()
agent = Agent()
gamma = 0.9
np.random.seed(0)

# 모든 𝑠∈𝑆,𝑎∈𝐴(𝑆)에 대해 초기화:
# 𝑄(𝑠,𝑎)←임의의 값
Q_table = np.random.rand(env.reward.shape[0], env.reward.shape[1],len(agent.action))

# Q(𝑡𝑒𝑟𝑚𝑖𝑛𝑎𝑙−𝑠𝑡𝑎𝑡𝑒,𝑎)=0
Q_table[2,2,:] = 0

max_episode = 10000
max_step = 100

print("start TD(0) control : Q-learning")
alpha = 0.1
epsilon = 0.8

# 각 에피소드에 대해 반복 :
for epi in tqdm(range(max_episode)):
    dleta = 0
    
    # S 를 초기화
    i = 0
    j = 0
    agent.set_pos([i,j])

    # 에피소드의 각 스텝에 대해 반복 :
    for k in range(max_step):
        
        # s 에서 행동 정책(Behavior policy)으로 행동 a를 선택 ( 예 : ε-greedy
        pos = agent.get_pos()
        action = e_greedy(Q_table,agent,epsilon)
        
        # 행동 a 를 취한 후 보상 r과 다음 상태 s^'를 관측
        observation, reward, done = env.move(agent, action)
        
        # ★SARSA와 차이점
        # s^' 에서 타깃 정책(Target policy)으로 행동 a^'를 선택 ( 예 : greedy)
        next_action = greedy(Q_table,agent,epsilon)
        
        # Q(s,a)←Q(s,a) + α[r+𝛾  maxa'𝑄(s',a')−𝑄(s,a)] 
        Q_table[pos[0],pos[1],action] += alpha * (reward + gamma * Q_table[observation[0], observation[1],next_action] - Q_table[pos[0],pos[1],action])
        
        # s가 마지막 상태라면 종료
        if done == True:
            break
        
# 학습된 정책에서 최적 행동 추출
optimal_policy = np.zeros((env.reward.shape[0], env.reward.shape[1]))
for i in range(env.reward.shape[0]):
    for j in range(env.reward.shape[1]):
        optimal_policy[i,j] = np.argmax(Q_table[i,j,:])

print("Q-learning : Q(s,a)")
show_q_table(np.round(Q_table,2),env)
print("Q-learning :optimal policy")
show_policy(optimal_policy,env)

start TD(0) control : Q-learning


100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 6362.98it/s]

Q-learning : Q(s,a)
+-----------------+-----------------+-----------------+
|      1.10       |      2.56       |      4.17       |
|  1.10      4.56 |  3.10      6.17 |  4.55      4.17 |
|      4.56       |      6.18       |      7.97       |
+-----------------+-----------------+-----------------+
|      3.10       |      4.56       |      6.17       |
|  2.56      6.18 |  4.56      7.98 |  6.18      5.97 |
|      6.18       |      7.97       |      9.97       |
+-----------------+-----------------+-----------------+
|      4.56       |      6.18       |      9.97       |
|  4.17      7.97 |  6.17      9.97 |  9.97      9.97 |
|      4.17       |      5.97       |      9.97       |
+-----------------+-----------------+-----------------+
Q-learning :optimal policy
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      ↓         |      ↓         |      ↓         |
|                 |                 |                 |
+---

# Env2: SARSA와 Q-Learning 비교하기

In [9]:
class Environment2():
    
    # 1. 미로밖(절벽), 길, 목적지와 보상 설정
    cliff = -3
    road = -1
    goal = 1
    
    # 2. 목적지 좌표 설정
    goal_position = [1,3]
    
    # 3. 보상 리스트 숫자
    reward_list = [[road,cliff,cliff, goal],
                   [road,road, road,  road],
                   [road,road, road,  road]]
    
    # 4. 보상 리스트 문자
    reward_list1 = [["road","cliff","cliff", "goal"],
                    ["road","road", "road", "road"],
                    ["road","road", "road", "road"]]
    
    # 5. 보상 리스트를 array로 설정
    def __init__(self):
        self.reward = np.asarray(self.reward_list)
        # np.asarray: np.array와 유사하지만, copy 후에 copy 데이터 변경 시 asarray는 원본도 변경

    # 6. 선택된 에이전트의 행동 결과 반환 (미로밖일 경우 이전 좌표로 다시 복귀)
    def move(self, agent, action):
        
        done = False
        
        # 6.1 행동에 따른 좌표 구하기
        new_pos = agent.pos + agent.action[action]
        
        # 6.2 현재좌표가 목적지 인지확인
        if self.reward_list1[agent.pos[0]][agent.pos[1]] == "goal":
            reward = self.goal
            observation = agent.set_pos(agent.pos)
            done = True
            
        # 6.3 이동 후 좌표가 미로 밖인 확인    
        elif self.reward_list1[agent.pos[0]][agent.pos[1]] == "cliff" or new_pos[0] < 0 or new_pos[0] >= self.reward.shape[0] or new_pos[1] < 0 or new_pos[1] >= self.reward.shape[1]:
            reward = self.cliff
            observation = agent.set_pos(agent.pos)
            done = True
            
        # 6.4 이동 후 좌표가 길이라면
        else:
            observation = agent.set_pos(new_pos)
            reward = self.reward[observation[0],observation[1]]
            
        return observation, reward, done

In [10]:
#ENV2 , SARSA

# TD(0) control : SARSA
np.random.seed(0)

# 환경, 에이전트를 초기화
env = Environment2()
agent = Agent()
gamma = 0.9

# 모든 𝑠∈𝑆,𝑎∈𝐴(𝑆)에 대해 초기화:
# 𝑄(𝑠,𝑎)←임의의 값
Q_table = np.random.rand(env.reward.shape[0], env.reward.shape[1],len(agent.action))

# Q(𝑡𝑒𝑟𝑚𝑖𝑛𝑎𝑙−𝑠𝑡𝑎𝑡𝑒,𝑎)=0
Q_table[2,2,:] = 0

max_episode = 10000
max_step = 100

print("start TD(0) control : SARSA")
alpha = 0.1
epsilon = 0.8

# 각 에피소드에 대해서 반복 :
for epi in tqdm(range(max_episode)):
    dleta = 0
    # S 를 초기화
    i = 0
    j = 0
    agent.set_pos([i,j])
    temp =0
    
    # s 에서 행동 정책(Behavior policy)으로 행동 a를 선택 ( 예 : ε-greedy
    action = e_greedy(Q_table,agent,epsilon)

    # 에피소드의 각 스텝에 대해서 반복 :
    for k in range(max_step):
        pos = agent.get_pos()
        
        # Q_visit_count[action, pos[0],pos[1]] +=1
        # 행동 a 를 취한 후 보상 r과 다음 상태 s^'  관측
        observation, reward, done = env.move(agent, action)

        # s^' 에서 타깃 정책(Target policy)으로 행동 a^'를 선택 ( 예 : ε-greedy
        next_action = e_greedy(Q_table,agent,epsilon)
        
        # Q(𝑆,𝐴)←Q(𝑆,𝐴) + α[𝑅+𝛾𝑄(𝑆',𝐴')−𝑄(𝑆,𝐴)]
        Q_table[pos[0],pos[1],action] += alpha * (reward + gamma * Q_table[observation[0], observation[1],next_action] - Q_table[pos[0],pos[1],action])

        # s←s^' ; a←a^'
        action = next_action
        
        # s가 마지막 상태라면 종료
        if done == True:
            break
        
# 학습된 정책에서 최적 행동 추출
optimal_policy = np.zeros((env.reward.shape[0], env.reward.shape[1]))
for i in range(env.reward.shape[0]):
    for j in range(env.reward.shape[1]):
        optimal_policy[i,j] = np.argmax(Q_table[i,j,:])

print("SARSA : Q(s,a)")
show_q_table(np.round(Q_table,2),env)
print("SARSA :optimal policy")
show_policy(optimal_policy,env)

start TD(0) control : SARSA


100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 5004.01it/s]

SARSA : Q(s,a)
+-----------------+-----------------+-----------------+-----------------+
|    -21.52       |    -29.96       |    -12.23       |      4.25       |
|-20.62    -29.96 |-29.96    -29.96 |-11.58    -11.42 |  4.20      4.58 |
|    -15.97       |    -29.96       |    -11.86       |      4.36       |
+-----------------+-----------------+-----------------+-----------------+
|    -20.49       |    -29.95       |    -13.15       |      4.81       |
|-17.31    -15.55 |-14.58    -10.28 |-15.33     -2.86 | -9.25     -5.61 |
|    -13.89       |    -11.50       |     -9.28       |     -6.70       |
+-----------------+-----------------+-----------------+-----------------+
|    -15.76       |    -14.80       |     -9.22       |     -3.51       |
|-15.35    -12.22 |-13.52     -9.99 |-11.80     -7.72 | -9.26     -8.81 |
|    -16.03       |    -14.35       |    -11.48       |     -9.30       |
+-----------------+-----------------+-----------------+-----------------+
SARSA :optimal policy
+

In [11]:
#ENV2 , Q-learning

#  TD(0) contro : Q-learning

# 환경, 에이전트를 초기화
env = Environment2()
agent = Agent()
gamma = 0.9
np.random.seed(0)

# 모든 𝑠∈𝑆,𝑎∈𝐴(𝑆)에 대해 초기화:
# 𝑄(𝑠,𝑎)←임의의 값
Q_table = np.random.rand(env.reward.shape[0], env.reward.shape[1],len(agent.action))

# Q(𝑡𝑒𝑟𝑚𝑖𝑛𝑎𝑙−𝑠𝑡𝑎𝑡𝑒,𝑎)=0
Q_table[2,2,:] = 0

max_episode = 10000
max_step = 100

print("start TD(0) control : Q-learning")
alpha = 0.1
epsilon = 0.8

# 각 에피소드에 대해 반복 :
for epi in tqdm(range(max_episode)):
    dleta = 0
    
    # S 를 초기화
    i = 0
    j = 0
    agent.set_pos([i,j])

    # 에피소드의 각 스텝에 대해 반복 :
    for k in range(max_step):
        
        # s 에서 행동 정책(Behavior policy)으로 행동 a를 선택 ( 예 : ε-greedy
        pos = agent.get_pos()
        action = e_greedy(Q_table,agent,epsilon)
        
        # 행동 a 를 취한 후 보상 r과 다음 상태 s^'를 관측
        observation, reward, done = env.move(agent, action)

        # s^' 에서 타깃 정책(Target policy)으로 행동 a^'를 선택 ( 예 : greedy)

        next_action = greedy(Q_table,agent,epsilon)
        
        # Q(s,a)←Q(s,a) + α[r+𝛾  maxa'𝑄(s',a')−𝑄(s,a)] 
        Q_table[pos[0],pos[1],action] += alpha * (reward + gamma * Q_table[observation[0], observation[1],next_action] - Q_table[pos[0],pos[1],action])
        
        # s가 마지막 상태라면 종료
        if done == True:
            break
        
# 학습된 정책에서 최적 행동 추출
optimal_policy = np.zeros((env.reward.shape[0], env.reward.shape[1]))
for i in range(env.reward.shape[0]):
    for j in range(env.reward.shape[1]):
        optimal_policy[i,j] = np.argmax(Q_table[i,j,:])

print("Q-learning : Q(s,a)")
show_q_table(np.round(Q_table,2),env)
print("Q-learning :optimal policy")
show_policy(optimal_policy,env)

start TD(0) control : Q-learning


100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 5452.38it/s]

Q-learning : Q(s,a)
+-----------------+-----------------+-----------------+-----------------+
|     -1.66       |    -29.95       |    -14.44       |      6.96       |
| -1.66    -29.95 |-29.95    -29.95 |-14.24    -14.25 |  7.35      7.49 |
|      1.50       |    -29.95       |    -14.29       |      7.22       |
+-----------------+-----------------+-----------------+-----------------+
|      0.34       |    -29.95       |    -15.37       |      7.64       |
| -0.51      2.79 |  1.48      4.22 |  2.75      5.83 |  4.06      3.59 |
|      0.32       |      1.45       |      2.69       |      3.81       |
+-----------------+-----------------+-----------------+-----------------+
|      1.47       |      2.74       |      4.17       |      5.58       |
| -1.73      1.42 |  0.28      2.61 |  1.22      3.80 |  1.95      1.44 |
|     -1.70       |     -0.65       |      0.51       |      1.57       |
+-----------------+-----------------+-----------------+-----------------+
Q-learning :optima